<a href="https://colab.research.google.com/github/FredLongo/DataMiningProject/blob/main/code/GetTwitterSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This first part is about processing the Twitter data to a format we can work with.   


# Project Overview

## Evaluation of Twiter stock sentement on price stock prices.

In this project we evaluate twiter tweet posts to see if tweets can act as leading indecators or laging indicaterst to stock prices.  We do this over a one day, three day, five day pre and post of tweet.  




# Aquiring Tweets

We targeted twitter tweets to data mine as a source for evaluation of sentament.  Twitter was chosen because many of the paper we reviewed took this approch.  We wanted to emulate some of their process.  However Twitter has changed it's policies over the past year to not allow twitter pulls for free to research.  As a result We needed to use a thired party product to pull a few months worth of tweets.  This third party was Apify (www.apify.com)  as a tool it can pull twiter feeds.  This file was created using the tool and the results of which can be found at # Data pulled from https://console.apify.com/actors/2s3kSMq7tpuC3bI6M/runs/cMp5HramffKLAmVSt#output

The json file is stored to the folloiwng location
 https://drive.google.com/file/d/1s429__b-2Gptf2oN2g-t--1aNPkMiyU-/view?usp=drive_link and should be downloaded to the local colab environment for processing.  


## Limiting/Filtering twitter-Tweets.json Data Set
During this process we thought about the diffrent approches to evaluation of stock.  We could look at all tweets and get sentenment with respect to the market as a whole.  However we found that their are so many ways to iterprete the tweets with respect to the market as a whole was difficalt. Some filtering was required.  We chose to limit our stocks to just the top 7 of the S&P 500.  By doing so we felt this this would be easerir to selectivly identify tweets with respect to the stock.  The stocks we chose are "Apple","Alphabet","Amazon","Microsoft","Meta","Nvidia" and "Tesla".  Using their Stock lable of "AAPL","GOOG","AMZN","MSFT","META","NVDA","TSLA"

We also just looked a a few months of tweets.  So as to limit the data set to just a sample timeframe.  

Twitter has many users(channels) that produce tweets. We recognize not all channels are intresed in producing relevent, relyable and consistent statements with respect to market or stocks.  So we limit the channals that we monitored to just a few key player that we though might be relevent in the marketspace of stock evaluations.  The channals we chose to evaluate are "@USNewsMoney", "@TMFStockAdvisor", "@ftfinancenews", "@Stocktwits", "@MadMoneyOnCNBC" and "@SquawkCNBC".  We found these to be popular that should fit our needs.




Loading tweet data.

In [68]:
import json
import csv
import requests
from datetime import datetime



In [95]:

#The Twitter-Tweets file can be pulled off the internet from the following

# As CSV
#Tweets_url = 'https://drive.google.com/file/d/1HqubufxP00MJtA_Rh75d3E0WXH-x9Pqt/view?usp=sharing'
#Tweets_filename = 'Tweets.csv'

# As JSON
#Tweets_url = 'https://drive.google.com/file/d/1s429__b-2Gptf2oN2g-t--1aNPkMiyU-/view?usp=drive_link'
Tweets_filename = 'twitter-tweets.json'

# As JSON Text File extention.
#Tweets_url = 'https://drive.google.com/file/d/1AFXhblSsY-5UmMEWNad2GHsVy_NRz4zu/view?usp=drive_link'


#infile = f"/content/{Tweets_filename}"
infile = Tweets_filename

print(infile)

with open(infile, "r") as f:
  data = json.load(f)






twitter-tweets.json


Load stock price data.

In [70]:

#Stock Price URL of the file to be downloaded
StockPrice_url = 'https://drive.google.com/file/d/1Hk3Ca66ai_vAO14EFAD1AfOEk2A60E-U/view?usp=drive_link'

# Send a GET request to the URL
StockPrice_response = requests.get(StockPrice_url)

# Ensure the request was successful
if StockPrice_response.status_code == 200:
    # Write the content of the response to a file
    with open('StockPrice.csv', 'wb') as file:
        file.write(StockPrice_response.content)
else:
    print(f"Failed to download the file. Status code: {StockPrice_response.status_code}")


# Data Structure

We needed a way to store our data so we implented SQLITE database to  use.


In [71]:

import sqlite3
conn = sqlite3.connect("Project.db")

# Create a cursor object using the cursor() method
cursor = conn.cursor()


In [74]:
# SQL command to create Tweets table

create_table_command = """
CREATE TABLE Tweets (
    Timestamp DATE,
    DateKey TEXT,
    Channel TEXT,
    Stock  TEXT,
    Neg_score FLOAT,
    Neu_score FLOAT,
    Pos_score FLOAT,
    Sentiment Int,
    Tweet_orginal TEXT,
    Tweet_clean TEXT

);
"""


# Execute the SQL command
cursor.execute(create_table_command)




In [73]:
#dbexec("drop table Tweets")

In [77]:
# SQL command to create Stocks table

create_table_command = """
CREATE TABLE Stocks (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Date DATE,
    DateKey TEXT,
    AAPL_C	FLOAT,
    AMZN_C	FLOAT,
    GOOG_C	FLOAT,
    META_C	FLOAT,
    MSFT_C	FLOAT,
    NVDA_C	FLOAT,
    TSLA_C	FLOAT,
    AAPL_H	FLOAT,
    AMZN_H	FLOAT,
    GOOG_H	FLOAT,
    META_H	FLOAT,
    MSFT_H	FLOAT,
    NVDA_H	FLOAT,
    TSLA_H	FLOAT,
    AAPL_L	FLOAT,
    AMZN_L	FLOAT,
    GOOG_L	FLOAT,
    META_L	FLOAT,
    MSFT_L	FLOAT,
    NVDA_L	FLOAT,
    TSLA_L	FLOAT,
    AAPL_O	FLOAT,
    AMZN_O	FLOAT,
    GOOG_O	FLOAT,
    META_O	FLOAT,
    MSFT_O	FLOAT,
    NVDA_O	FLOAT,
    TSLA_O	FLOAT,
    AAPL_V	FLOAT,
    AMZN_V	FLOAT,
    GOOG_V	FLOAT,
    META_V	FLOAT,
    MSFT_V	FLOAT,
    NVDA_V	FLOAT,
    TSLA_V	FLOAT,
    AAPL_N	FLOAT,
    AMZN_N	FLOAT,
    GOOG_N	FLOAT,
    META_N	FLOAT,
    MSFT_N	FLOAT,
    NVDA_N	FLOAT,
    TSLA_N	FLOAT,
    AAPL_U	FLOAT,
    AMZN_U	FLOAT,
    GOOG_U	FLOAT,
    META_U	FLOAT,
    MSFT_U	FLOAT,
    NVDA_U	FLOAT,
    TSLA_U	FLOAT,
    AAPL_P	FLOAT,
    AMZN_P	FLOAT,
    GOOG_P	FLOAT,
    META_P	FLOAT,
    MSFT_P	FLOAT,
    NVDA_P	FLOAT,
    TSLA_P  FLOAT
);
"""

# Execute the SQL command
cursor.execute(create_table_command)

In [76]:
#dbexec("drop table Stocks")

In [96]:
# List all table in DB

list_tables_command = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(list_tables_command)

tables = cursor.fetchall()
for table in tables:
  print(table[0])

sqlite_sequence
Tweets
Stocks


In [97]:

# SQL command to get all columns of a table

table_name = 'Tweets'  # Replace with your table name
pragma_query = f"PRAGMA table_info({table_name});"

# Execute the command
cursor.execute(pragma_query)

# Fetch all results
columns = cursor.fetchall()

# Print column names
print(f"Table: {table_name}")
for col in columns:
    print(col[1])  # Column names are in the second position in the result tuple
print("")


table_name = 'Stocks'  # Replace with your table name
pragma_query = f"PRAGMA table_info({table_name});"

# Execute the command
cursor.execute(pragma_query)

# Fetch all results
columns = cursor.fetchall()

# Print column names
print(f"Table: {table_name}")
for col in columns:
    print(col[1])  # Column names are in the second position in the result tuple




Table: Tweets
Timestamp
DateKey
Channel
Stock
Neg_score
Neu_score
Pos_score
Sentiment
Tweet_orginal
Tweet_clean

Table: Stocks
ID
Date
DateKey
AAPL_C
AMZN_C
GOOG_C
META_C
MSFT_C
NVDA_C
TSLA_C
AAPL_H
AMZN_H
GOOG_H
META_H
MSFT_H
NVDA_H
TSLA_H
AAPL_L
AMZN_L
GOOG_L
META_L
MSFT_L
NVDA_L
TSLA_L
AAPL_O
AMZN_O
GOOG_O
META_O
MSFT_O
NVDA_O
TSLA_O
AAPL_V
AMZN_V
GOOG_V
META_V
MSFT_V
NVDA_V
TSLA_V
AAPL_N
AMZN_N
GOOG_N
META_N
MSFT_N
NVDA_N
TSLA_N
AAPL_U
AMZN_U
GOOG_U
META_U
MSFT_U
NVDA_U
TSLA_U
AAPL_P
AMZN_P
GOOG_P
META_P
MSFT_P
NVDA_P
TSLA_P


In [80]:
def dbexec(command):
  # Execute the command
  cursor.execute(command)

  # Fetch all the rows from the executed query
  rows = cursor.fetchall()

  # Iterate through rows and print them
  for row in rows:
    print(row)




In [81]:

def Stock_case_AAPL():
    return "AAPL"

def Stock_case_GOOG():
    return "GOOG"

def Stock_case_AMZN():
    return "AMZN"

def Stock_case_META():
    return "META"

def Stock_case_MSFT():
    return "MSFT"

def Stock_case_NVDA():
    return "NVDA"

def Stock_case_TSLA():
    return "TSLA"

def Stock_case_default():
    return "ERROR"

def Stock_Symble(StockID):
  StockID = StockID.strip()
  switcher = {
      'AAPL':       Stock_case_AAPL,
      'Alphabet':  	Stock_case_GOOG,
      'AMZN':   		Stock_case_AMZN,
      'Apple':   		Stock_case_AAPL,
      'GOOG':   		Stock_case_GOOG,
      'META':   	 	Stock_case_META,
      'Microsoft': 	Stock_case_MSFT,
      'MSFT':   		Stock_case_MSFT,
      'NVDA':   		Stock_case_NVDA,
      'Nvidia':   	Stock_case_NVDA,
      'Tesla':   		Stock_case_TSLA,
      'TSLA':   		Stock_case_TSLA
  }

  # Get the function from switcher dictionary
  case_function = switcher.get(StockID, Stock_case_default)

  # Execute the function
  return case_function()




# Evaluation of Twitter Sentiment

At this point I want to add the code to do the sentiment alalisis.



In [82]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Negative = 0
Neutral = 1
Positive = 2



In [83]:
def remove_quotes(input_string):
    # Remove single quotes
    no_single_quotes = input_string.replace("'", "")

    # Remove double quotes
    no_quotes = no_single_quotes.replace('"', "")

    return no_quotes

In [84]:
def remove_emojis(text):
    # Define the emoji pattern using Unicode ranges
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    # Remove emojis from the text
    return emoji_pattern.sub(r'', text)


In [85]:
import re

def clean_string(input_string):
    # Regex to match only visible characters
    # This excludes most control characters like \n, \t, \r, etc.
    visible_chars_regex = r'[^\x20-\x7E]'

    # Substitute non-visible characters with an empty string
    cleaned_string = re.sub(visible_chars_regex, '', input_string)

    return cleaned_string


In [86]:
def preprocesses_tweet(tweet):
  tweet_words = []

  cleaned_string = remove_emojis(tweet)
  cleaned_string = remove_quotes(cleaned_string)
  cleaned_string = clean_string(cleaned_string)


  # replace
  for word in cleaned_string.split(' '):
    # username
    if word.startswith('@') and len(word) > 1 :
      word = '@user'

    # http link
    elif word.startswith('http'):
      word = "http"

    elif word == '':
      continue

    tweet_words.append(word)


  tweet_proc = " ".join(tweet_words)
  return tweet_proc


In [87]:
def get_tweet_sentiment(tweet_proc):
  # sentiment analysis
  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
  # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
  output = model(**encoded_tweet)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  maxposition = 0
  maxscore = 0

  for i in range(len(scores)):
    if maxscore <= scores[i]:
      maxposition = i
      maxscore = scores[i]

  return labels[maxposition]  , scores[maxposition], maxposition, scores[0],scores[1],scores[2]




In [104]:
# Evaluate Tweet and and to database.
import time

#out_data = []
loop_count = -1
tweet_proc = ""


# Need to create a tweet loop

for x in data:
  loop_count+=1
  print(loop_count)
  try:

    tweet_proc = preprocesses_tweet(x['text'])
    sentiment_label, sentiment_score, Sentiment, scores0,scores1,scores2 = get_tweet_sentiment(tweet_proc)
    stock_symble_o = x['searchQuery']
    stock_symble = Stock_Symble(x['searchQuery'])


    date_object = datetime.strptime(x['timestamp'],  "%Y-%m-%dT%H:%M:%S.%fZ" )
    date_key = date_object.strftime("%Y-%m-%d")

    insert_command = f"INSERT INTO Tweets (Timestamp, DateKey, Channel, Stock, Neg_score, Neu_score, Pos_score, sentiment,Tweet_clean) VALUES ('{x['timestamp']}','{date_key}','{x['user']['username']}', '{stock_symble}','{scores0}','{scores1}','{scores2}','{Sentiment}','{tweet_proc}')"
    #insert_command = f"INSERT INTO Tweets (Timestamp, Channel, Stock, Neg_score, Neu_score, Pos_score, sentiment,Tweet_clean) VALUES ('{x['timestamp']}','{x['user']['username']}', '{stock_symble}','{scores0}','{scores1}','{scores2}','{Sentiment}','{tweet_proc}')"
    cursor.execute(insert_command)

  except Exception as e:
    print(f"---------------------------------------")
    print(f"exception on {str(loop_count)}")
    print(f"Error {e}")
    continue



# Commit the changes to database on drive.
conn.commit()

dbexec("Select count(1) from Tweets")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [103]:
cursor.execute("DELETE FROM Tweets")

In [89]:
dbexec("Select Stock , count(1) from Tweets group by Stock ")
#dbexec("Select count(1) from Tweets where Stock = 'ERROR' ")
#dbexec("Select  * from Tweets where Stock = 'ERROR' ")
#dbexec("Select   Timestamp, Channel, Stock, Neg_score, Neu_score, Pos_score, Sentiment, Tweet_orginal, Tweet_clean  from Tweets where sentiment = 0")


('AAPL', 52)
('AMZN', 39)
('GOOG', 8)
('META', 39)
('MSFT', 36)
('NVDA', 55)
('TSLA', 63)


In [ ]:


'''
csv_file_name = infile[:-5] + "_out.csv"

# Open the CSV file in write mode
with open(csv_file_name, mode='w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header row based on the keys in the JSON data
    header = out_data[0].keys()
    csv_writer.writerow(header)

    # Write each row of data from the JSON object
    for row in out_data:
        csv_writer.writerow(row.values())

'''

# Aquiring Stock Prices

In [90]:
from pickle import FALSE
# Load the CSV file we downloaded.  It contains the Stock Prices for the past few months.
import csv

# Specify the CSV file name
csv_file_name = "StockPrices.csv"

# Initialize an empty list to store the data
data = []

# Open the CSV file in read mode
with open(csv_file_name, mode='r', newline='') as csv_file:
    # Create a CSV reader
    csv_reader = csv.reader(csv_file)

    # Read the header row
    header = next(csv_reader)

    header_row = True


    # Read the data rows
    for row in csv_reader:
        #data.append(row)

        if header_row == True:
          header_row = False
          continue
        date_object = datetime.strptime(row[0], "%m/%d/%Y")
        date_key = date_object.strftime("%Y-%m-%d")
        insert_command = f'''INSERT INTO Stocks
                        ( Date,DateKey,
                          AAPL_C,	AMZN_C,	GOOG_C,	META_C,	MSFT_C,	NVDA_C,	TSLA_C,
                          AAPL_H,	AMZN_H,	GOOG_H,	META_H,	MSFT_H,	NVDA_H,	TSLA_H,
                          AAPL_L,	AMZN_L,	GOOG_L,	META_L,	MSFT_L,	NVDA_L,	TSLA_L,
                          AAPL_O,	AMZN_O,	GOOG_O,	META_O,	MSFT_O,	NVDA_O,	TSLA_O,
                          AAPL_V,	AMZN_V,	GOOG_V,	META_V,	MSFT_V,	NVDA_V,	TSLA_V
                        )
                      VALUES
                        (
                          '{date_object}','{date_key}',
                          '{row[1]}',	'{row[2]}',	  '{row[3]}',	  '{row[4]}',	  '{row[5]}',	  '{row[6]}',   '{row[7]}',
                          '{row[15]}',	'{row[16]}',	'{row[17]}',	'{row[18]}',	'{row[19]}',	'{row[20]}',	'{row[21]}',
                          '{row[22]}',	'{row[23]}',	'{row[24]}',	'{row[25]}',	'{row[26]}',	'{row[27]}',	'{row[28]}',
                          '{row[29]}',	'{row[30]}',	'{row[31]}',	'{row[32]}',	'{row[33]}',	'{row[34]}',	'{row[35]}',
                          '{row[43]}',	'{row[44]}',	'{row[45]}',	'{row[46]}',	'{row[47]}',	'{row[48]}',	'{row[49]}'
                        )'''
        cursor.execute(insert_command)


# Display the loaded data
#for row in data:
#    print(row)


dbexec("Select count(1) from Stocks")


(128,)


In [47]:
#dbexec("DELETE FROM Stocks ")


In [105]:
dbexec("Select  * from tweets  LIMIT 10")
dbexec("Select  * from Stocks  LIMIT 10")

('2023-10-17T13:21:00.000Z', '2023-10-17', '@Stocktwits', 'NVDA', 0.47528815269470215, 0.4988061189651489, 0.025905678048729897, 1, None, 'Nvidia stock put on slides this morning after reports drop that the Biden administration is tightening restrictions on Chinas ability to buy advanced semiconductors.$NVDA -3.75%')
('2023-10-27T13:47:00.000Z', '2023-10-27', '@Stocktwits', 'AMZN', 0.004496750887483358, 0.8170449733734131, 0.17845827341079712, 1, None, 'Top Trending stocks & crypto on Stocktwits this morning:1 $AMZN 2 $SQ3 $INTC4 $F5 $ENPH6 $CRSP7 $SNY8 $DXCM9 $ABBV $CPRX')
('2023-10-25T13:47:00.000Z', '2023-10-25', '@Stocktwits', 'MSFT', 0.004904467146843672, 0.7698082327842712, 0.2252873033285141, 1, None, 'Top Trending stocks & crypto on Stocktwits this morning:1 $MSFT2 $GOOGL 3 $ASPA4 $AVXL5 $SNAP6 $GOOG7 $BA8 $PYPL9 $TDOC $PEPE')
('2023-10-24T21:12:00.000Z', '2023-10-24', '@Stocktwits', 'MSFT', 0.004431292414665222, 0.7569287419319153, 0.23864008486270905, 1, None, 'Top Trending s

In [128]:
# Update Stocks table with avages

update_query = '''
    UPDATE Stocks
    SET AAPL_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AAPL'
    ),
    AAPL_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AAPL'
    ),
    AAPL_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AAPL'
    ),
    AMZN_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AMZN'
    ),
    AAPL_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AMZN'
    ),
    AAPL_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'AMZN'
    ),
    GOOG_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'GOOG'
    ),
    GOOG_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'GOOG'
    ),
    GOOG_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'GOOG'
    ),
    META_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'META'
    ),
    META_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'META'
    ),
    META_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'META'
    ),
    MSFT_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'MSFT'
    ),
    MSFT_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'MSFT'
    ),
    MSFT_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'MSFT'
    ),
    NVDA_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'NVDA'
    ),
    NVDA_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'NVDA'
    ),
    NVDA_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'NVDA'
    ),
    TSLA_N = (
        SELECT avg(Neg_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'TSLA'
    ),
    TSLA_U = (
        SELECT avg(Neu_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'TSLA'
    ),
    TSLA_P = (
        SELECT avg(Pos_score)
        FROM Tweets as t
        WHERE Stocks.DateKey = t.DateKey AND t.Stock = 'TSLA'
    );
'''

dbexec(update_query)




In [129]:

#check

update_query = '''
    select
    AAPL_N,	AMZN_N,	GOOG_N,	META_N,	MSFT_N,	NVDA_N,	TSLA_N,
	  AAPL_U,	AMZN_U,	GOOG_U,	META_U,	MSFT_U,	NVDA_U,	TSLA_U,
	  AAPL_P,	AMZN_P,	GOOG_P,	META_P,	MSFT_P,	NVDA_P,	TSLA_P
    from Stocks
    ;
'''
dbexec(update_query)



(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

# Joining Stock Prices with respect to Tweets

# Analytics

# Conclution